In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle
import time

In [2]:
URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

BOWlist = soup.find_all('a')

URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Create a list with links
BOWpages = ['https://birdsoftheworld.org' + pages.get('href').replace('introduction', '') for pages in BOWlist 
                   if pages.get('href') != None
                   if pages.get('href').startswith('/bow/species')]

In [30]:
BOWpages.index('https://birdsoftheworld.org/bow/species/dacgos1/cur/') 

3100

In [4]:
# Init driver
browser = webdriver.Safari()

In [5]:
# Login URL
browser.get("https://secure.birds.cornell.edu/cassso/login?service=https%3A%2F%2Fbirdsoftheworld.org%2Flogin%2Fcas")

# Username
username = browser.find_element_by_id("input-user-name")
username.clear()
username.send_keys("RobertvdV")

# Password
password = browser.find_element_by_name("password")
password.clear()
password.send_keys("XXXXXXXXXX")

# Sign in
SignIn = browser.find_element_by_id("form-submit")
SignIn.send_keys(u'\ue007')

In [32]:
browser.get(BOWpages[10566] + 'appearance')
time.sleep(2.5)


In [33]:
HTML = browser.page_source
soup = BeautifulSoup(HTML, 'html.parser')

In [34]:
# Init dict
DataBOW = collections.defaultdict(list)

# Reference Remover
ReferenceRemover1 = ' \(\d+\n\t.+?\n\t\n\t\tClose\n\t\n\)'
ReferenceRemover2 = ' \[\d+\n\t.+?\n\t\n\t\tClose\n\t\n\]'
# Brackets Remover
BracketsRemover = ' \((.*?)\)'

# Bird name 
if len(soup.title.text.split(' - ')) == 4:

    BirdName = soup.title\
                    .text\
                    .split(' - ')[1]\
                    .lstrip()
else:
    BirdName = soup.title\
                .text\
                .split(' - ')[0]\
                .lstrip()

# Find all text data
Paragraphs = soup.find_all('p')

# Loop over the text
for Para in Paragraphs[1:]:
    try:
        
        # Check to which section the text belongs
        SubSection = Para.find_previous_siblings()[0]\
                         .text\
                         .rstrip()\
                         .lstrip()
        
        # Check to which section the text belongs
        SubSubSection = Para.find_previous_siblings()[-1]\
                             .text\
                             .rstrip()\
                             .lstrip()
        
    # If a text does not have a header, skip    
    except:
        continue
    
    if 'Formative' in SubSection:
        continue
    if 'Juvenile' in SubSection:
        continue
    
    # See which are needed and append with 1
    if SubSection == 'Identification' or\
       SubSection == 'Adult' or\
       SubSubSection == 'Plumages' or\
       SubSubSection == 'Bare Parts':
          
        # Clean the text from references (NEEDS EDIT)    
        TextCleaned = re.sub(ReferenceRemover1, '', Para.text, re.DOTALL)
        TextCleaned = re.sub(ReferenceRemover2, '', TextCleaned, re.DOTALL)
        TextCleaned = re.sub(BracketsRemover, '', TextCleaned)
        
        # Drop useless short nonsense
        if len(TextCleaned.split()) <2:
            continue
        else:
            DataBOW[BirdName] += (1, TextCleaned)
            
    # Otherwise label 2        
    else:
        # Clean the text from references (NEEDS EDIT)    
        TextCleaned = re.sub(ReferenceRemover1, '', Para.text, re.DOTALL)
        TextCleaned = re.sub(ReferenceRemover2, '', TextCleaned, re.DOTALL)
        TextCleaned = re.sub(BracketsRemover, '', TextCleaned)
        
        # Drop useless short nonsense
        if len(TextCleaned.split()) <2:
            continue
        else:
            DataBOW[BirdName] += (0, TextCleaned)

In [35]:
DataBOW

defaultdict(list,
            {'Northern Cardinal': [0,
              'You are currently viewing one of the free accounts available in our complimentary tour of Birds of the World. In this courtesy review, you can access all the life history articles and the multimedia galleries associated with this account.',
              0,
              'For complete access to all accounts, a subscription is required.',
              0,
              'Subscribe Now',
              1,
              'Medium-sized songbird; length of C. c. cardinalis, male 22.2–23.5 cm, female 20.9–21.6 cm. Average mass of C. c. cardinalis 42–48 g, though there appears to be variation with geographic location and time of year. Plumage distinctive—adult male in Definitive Basic is red with black face mask surrounding bill; female in Definitive Basic is grayish tan, with some red on wings, tail, crest, face, and breast, red to red-orange underwings, and pale gray to black face mask. Both sexes have a prominent crest and

In [22]:
BirdName

'Northern Cardinal'

In [16]:
soup.title.text.split(' - ')

['\t \n\t    Spotted Eagle-Owl',
 'Bubo africanus',
 '\n\t             Birds of the World']

In [24]:
len(soup.title.text.split(' - '))

4